In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
from src.metrics import *
from settings import *
from src.data import generate

instruments = ['Guitar']
instruments_aug = ['Accordion', 'Violin', 'Piano']

datasets_raw = [pd.read_pickle(os.path.join(METADATA_DIR_PROCESSED, f'data_{instrument.lower()}.pkl')) for instrument in instruments]
datasets_augmented = [pd.read_pickle(os.path.join(METADATA_DIR_AUGMENTED_RAW, f'data_{instrument.lower()}.pkl')) for instrument in instruments_aug]

from src.data.preprocessing import get_max_shape
max_spectrogram_size = max(map(lambda df: get_max_shape(df), datasets_raw+datasets_augmented))
max_spectrogram_size

from src.data.preprocessing import uniform_shape
uniform = lambda df: uniform_shape(df, max_spectrogram_size)

datasets_raw = list(map(uniform,datasets_raw))
datasets_augmented = list(map(uniform,datasets_augmented))
datasets_augmented = list(map(lambda df: df[['spectrogram','class_ID', 'class_name','augmentation']],datasets_augmented))
datasets_augmented = list(map(lambda df: df.reset_index(drop=True), datasets_augmented))

import numpy as np
#spectrograms_raw = [[x for x in dataset['spectrogram']] for dataset in datasets_raw]
#spectrograms_augmented = [[x for x in dataset['spectrogram']] for dataset in datasets_augmented]
#all_spectrograms = np.concatenate(spectrograms_raw + spectrograms_augmented)

#all_spectrograms_flattened = all_spectrograms.flatten()

2023-05-19 11:01:46.953420: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-19 11:01:47.078399: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-19 11:01:47.733663: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.0: cannot open shared object file: No such file or directory
2023-05-19 11:01:47.733743: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64]

In [6]:
class StatsRecorder:
    def __init__(self, data=None):
        """
        data: ndarray, shape (nobservations, ndimensions)
        """
        if data is not None:
            data = np.atleast_2d(data)
            self.mean = data.mean(axis=0)
            self.std  = data.std(axis=0)
            self.nobservations = data.shape[0]
            self.ndimensions   = data.shape[1]
        else:
            self.nobservations = 0

    def update(self, data):
        """
        data: ndarray, shape (nobservations, ndimensions)
        """
        if self.nobservations == 0:
            self.__init__(data)
        else:
            data = np.atleast_2d(data)
            if data.shape[1] != self.ndimensions:
                raise ValueError("Data dims don't match prev observations.")

            newmean = data.mean(axis=0)
            newstd  = data.std(axis=0)

            m = self.nobservations * 1.0
            n = data.shape[0]

            tmp = self.mean

            self.mean = m/(m+n)*tmp + n/(m+n)*newmean
            self.std  = m/(m+n)*self.std**2 + n/(m+n)*newstd**2 +\
                        m*n/(m+n)**2 * (tmp - newmean)**2
            self.std  = np.sqrt(self.std)

            self.nobservations += n

In [45]:
train_normalized['spectrogram'].reset_index(drop=True).describe()

In [42]:
from src.metrics import *
from settings import *
from src.data import generate
from src.processing import *
from src.model import CNN
from src.data.preprocessing import get_most_shape
from setup_logging import setup_logging

from scipy.stats import boxcox
from scipy.stats import boxcox_normmax

train_datas = []
test_datas = []

for dataset in datasets_raw:
    train_data, test_data = train_test_split(dataset, augmented=False, split_ratio=0.65)
    train_datas.append(train_data)
    test_datas.append(test_data)

for dataset in datasets_augmented:
    train_data, test_data = train_test_split(dataset, augmented=False, split_ratio=0.65)
    train_datas.append(train_data)
    test_datas.append(test_data)

train_data = pd.concat(train_datas)
test_data = pd.concat(test_datas)

spectrograms_train = np.array([x for x in train_data['spectrogram']])
curr_shape = spectrograms_train.shape
spectrograms_train_squashed = spectrograms_train.reshape(curr_shape[0]*curr_shape[1], curr_shape[2])

means = np.mean(spectrograms_train_squashed, axis=0)
stds = np.std(spectrograms_train_squashed, axis=0)
#lmbdas = np.apply_along_axis(lambda col: boxcox_normmax(col + 1), 0, spectrograms_train_squashed)
#print('finished calculating lambdas')
#lmbdas = np.apply_along_axis(lambda col: col.shape, 0, spectrograms_train_squashed)
#print(lmbdas.shape)
# global_stats  = StatsRecorder()

# for index, row in train_data.iterrows():
#     global_stats.update(row['spectrogram'])

# normalize = lambda x: (x-global_stats.mean)/global_stats.std
# def normalize(spectro):
#     axis = 1
#     Ni, Nk = a.shape[:axis], a.shape[axis+1:]
#     for ii in np.ndindex(Ni):
#         for kk in np.ndindex(Nk):
#             f = func1d(arr[ii + np.s_[:,] + kk])
#             Nj = f.shape
#             for jj in ndindex(Nj):
#                 out[ii + jj + kk] = f[jj]
i=0
def normalize_by_column(col):
    global i
    tmp = boxcox(x=col + 1, lmbda=lmbdas[i])
    i+=1
    return tmp

def normalize_boxcox(spectro):
    global i
    i=0
    #print(spectro.shape)
    tmp = np.apply_along_axis(normalize_by_column, 0, spectro)
    #print(tmp.shape)
    return tmp

def normalize_mean_std(spectro):
    return (spectro - means) / stds

def ret_normalized(df):
    return pd.concat((df.loc[:,["class_ID","class_name"]], df['spectrogram'].apply(normalize_mean_std, 1)), axis=1)

train_normalized = ret_normalized(train_data)
test_normalized = ret_normalized(test_data)

most_shape = get_most_shape(train_normalized)

X_train, y_train = features_target_split(train_normalized)
X_test, y_test = features_target_split(test_normalized)

X_train, X_test = reshape_feature_CNN(X_train, size=max_spectrogram_size), reshape_feature_CNN(X_test, size=max_spectrogram_size)

y_test_values = y_test.copy()

# One-Hot encoding for classes
y_train, y_test = one_hot_encode(y_train), one_hot_encode(y_test)

# Instance of CNN model
cnn = CNN(most_shape)

cnn.train(X_train, y_train, X_test, y_test)
#cnn.evaluate(X_train, y_train, X_test, y_test)

src.processing - INFO - Start train test split with split ratio: 0.65
src.processing - INFO - Number of training samples is 1300
src.processing - INFO - Number of testing samples is 700
src.processing - INFO - Train test split completed
src.processing - INFO - Start train test split with split ratio: 0.65
src.processing - INFO - Number of training samples is 585
src.processing - INFO - Number of testing samples is 315
src.processing - INFO - Train test split completed
src.processing - INFO - Start train test split with split ratio: 0.65
src.processing - INFO - Number of training samples is 585
src.processing - INFO - Number of testing samples is 315
src.processing - INFO - Train test split completed
src.processing - INFO - Start train test split with split ratio: 0.65
src.processing - INFO - Number of training samples is 585
src.processing - INFO - Number of testing samples is 315
src.processing - INFO - Train test split completed
The most frequent shape is (128, 107)
src.processing - 

In [10]:
lmbdas.shape

(107,)

In [ ]:
test_instruments = instruments + instruments_aug
for test_data, instrument in zip(test_datas, test_instruments):
    X_test = test_data['spectrogram']
    X_test = np.array([x.reshape( (128, max_spectrogram_size, 1) ) for x in X_test])
    y_test = test_data['class_ID']

    y_test_values=y_test
    y_test = np.array(keras.utils.to_categorical(y_test, 10))

    score = cnn.model.evaluate(X_test,y_test)
    print(f'Test score for instrument: {instrument}')
    print('\tTest loss:', score[0])
    print('\tTest accuracy:', score[1])
    print('\tTest precision:', score[2])
    print('\tTest recall:', score[3])
    print('\tTest f1-score:', score[4])